In [96]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
import glove_solution
import matplotlib.pyplot as plt

In [97]:
all_words = np.load('embeddings.npy')

In [98]:
all_words

array([[ 0.12667778,  0.25033326,  0.17753004, ..., -0.05073233,
         0.40251133,  0.3292477 ],
       [ 0.18150658,  0.34523058,  0.23116592, ..., -0.08196746,
         0.53457013,  0.41001274],
       [ 0.16273848,  0.3311124 ,  0.23332552, ..., -0.09192156,
         0.59256165,  0.45893371],
       ...,
       [ 1.03691048,  1.52507158,  0.1381153 , ...,  1.5792987 ,
         1.57201574, -1.02772805],
       [ 0.61629264, -2.44090437, -2.08893546, ...,  0.28810867,
        -0.53614846, -1.7232222 ],
       [ 2.14907153,  1.90476016, -1.0027138 , ...,  1.07865881,
        -0.84661144, -1.5454268 ]])

In [99]:
# Import vocab and transform it into an array
vocab_cut = pd.read_fwf('vocab_cut.txt', sep = "\t", header = None, error_bad_lines=False)
vocab_cut.columns = ['word']
voc = vocab_cut.values
arg = np.argwhere('!')
voc

array([['<user>'],
       ['!'],
       ['i'],
       ...,
       ['#1dson'],
       ['#18'],
       ['#10fac']], dtype=object)

In [100]:
np.argwhere(voc == 'hi').shape

(1, 2)

In [101]:
# Load train set positiv and train set negativ
tweet_pos = pd.read_csv('data/train_pos.txt', sep = '\t', header = None)
tweet_neg = pd.read_csv('data/train_neg.txt', sep = '\t', header = None)
tweet_test = pd.read_csv('data/test_data.txt', sep = '\t', header = None)

In [102]:
tweet_pos.head()

,0
0,<user> i dunno justin read my mention or not ....
1,"because your logic is so dumb , i won't even c..."
2,<user> just put casper in a box ! looved the...
3,<user> <user> thanks sir > > don't trip lil ma...
4,visiting my brother tmr is the bestest birthda...


In [103]:
tweet_neg.head()

,0
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...


In [104]:
tweet_test.head()

,0
0,"1,sea doo pro sea scooter ( sports with the po..."
1,"2,<user> shucks well i work all week so now i ..."
2,"3,i cant stay away from bug thats my baby"
3,"4,<user> no ma'am ! ! ! lol im perfectly fine ..."
4,"5,whenever i fall asleep watching the tv , i a..."


In [105]:
import re

In [106]:
def tweet_to_vect(tweet):
    word_array = re.findall(r'\w+', tweet)
    vect = np.zeros(20)
    for word in word_array:
        i = np.argwhere(voc == word)
        if(i.shape[0] != 0): 
            vect += all_words[i[0][0]]
    return vect/len(word_array)

In [107]:
tweet_to_vect('i love machine learning')

array([ 0.08483506,  0.18905336,  0.13301682,  0.28761705,  0.15026169,
        0.16017279, -0.12708856, -0.16646328, -0.28536286,  0.23276146,
       -0.03472253, -0.06886545, -0.33791882, -0.17811641, -0.39209849,
       -0.05214223,  0.27099058, -0.06285391,  0.33469796,  0.25988553])

In [110]:
tweet_pos['vect'] = ""
tweet_pos['pred'] = 1
tweet_neg['vect'] = ""
tweet_neg['pred'] = -1

tweet_pos.columns = ['tweet', 'vect', 'pred']
tweet_neg.columns = ['tweet', 'vect', 'pred']

In [113]:
tweet_pos.vect = tweet_pos.tweet.apply(lambda x: tweet_to_vect(x))
tweet_pos

,tweet,vect,pred
0,<user> i dunno justin read my mention or not ....,"[0.12187350126213467, 0.2392204874081336, 0.17...",1
1,"because your logic is so dumb , i won't even c...","[0.029361259698287817, 0.2845059508864009, 0.1...",1
2,<user> just put casper in a box ! looved the...,"[0.03723710238158048, 0.010057370590752113, 0....",1
3,<user> <user> thanks sir > > don't trip lil ma...,"[0.01164429224360294, -0.1313781541898918, 0.1...",1
4,visiting my brother tmr is the bestest birthda...,"[0.12157701280012964, 0.07784914951046543, 0.0...",1
5,<user> yay ! ! #lifecompleted . tweet / facebo...,"[0.08271660403213885, 0.15320062263572212, 0.2...",1
6,<user> #1dnextalbumtitle : feel for you / roll...,"[-0.033055780925191285, 0.10938178560113726, 0...",1
7,workin hard or hardly workin rt <user> at hard...,"[-0.02979380671545447, 0.11863781919389324, 0....",1
8,<user> i saw . i'll be replying in a bit .,"[0.016231694326029035, 0.23733636098281602, 0....",1
9,this is were i belong,"[-0.05273754339083074, 0.25581083918924435, 0....",1


In [ ]:
tweet_neg.vect = tweet_neg.tweet.apply(lambda x: tweet_to_vect(x))